In [2]:
import os, numpy as np, pandas as pd, cv2, glob, matplotlib.pyplot as plt, re, tifffile as tiff
from skimage import measure
from scipy.stats import ttest_ind, mannwhitneyu
from scipy.spatial.distance import pdist
from pathlib import Path

In [5]:
def get_number(filename):
    match = re.search(r'(\d+)', filename)
    return match.group(1) if match else None

def filter_small_components(mask, fraction=0.1):
    labeled = measure.label(mask, connectivity=2)
    props_df = pd.DataFrame(
        measure.regionprops_table(
            labeled, properties=("label", "area", "centroid")
        )
    )
    if props_df.empty:
        return mask.astype(mask.dtype), props_df

    # Find cutoff size so that ~fraction of components are in the bottom group
    sizes = props_df["area"].values
    unique_sizes = np.unique(sizes)
    cutoff_index = int(np.floor(len(sizes) * fraction))
    if cutoff_index <= 0:
        return mask.astype(mask.dtype), props_df

    sorted_sizes = np.sort(sizes)
    cutoff_size = sorted_sizes[cutoff_index]
    cutoff_size = max(unique_sizes[unique_sizes <= cutoff_size])

    # Keep only components larger than cutoff_size
    keep_labels = props_df.loc[props_df["area"] > cutoff_size, "label"].values
    filtered_mask = np.isin(labeled, keep_labels).astype(mask.dtype)
    filtered_props_df = props_df[props_df["label"].isin(keep_labels)].reset_index(drop=True)

    print("Total regions detected:", len(props_df))
    print("Regions after filtering:", len(filtered_props_df))

    return filtered_mask, filtered_props_df.drop("label", axis = 1)



def seg_droplets(root_path, img_path):
    lipid = cv2.imread(os.path.join(root_path, img_path), cv2.IMREAD_UNCHANGED)
    top = 1.0
    thresh = np.percentile(lipid, [100-top, 100])
    seg_lipid = np.where(lipid < thresh[0], 0, 255)
    mask, props_df = filter_small_components(seg_lipid, fraction=0.9)

    df_name = os.path.join(root_path, f"roi{get_number(img_path)}_lipid_props.csv")

    # Save mask image
    # cv2.imwrite(os.path.join(root_path, f"roi{get_number(img_path)}_seg_lipid.png"), mask * 255)
    # props_df.to_csv(os.path.join(root_path, f"roi{get_number(img_path)}_lipid_props.csv"), index=False)

    # Overlay masked regions on original image with translucent red color
    overlay_path = os.path.join(root_path, f"roi{get_number(img_path)}_seg_lipid_overlay.png")
    plt.figure(figsize=(6, 6))
    plt.imshow(lipid, cmap="gray")
    # Use a red colormap for the mask overlay
    plt.imshow(mask, cmap="Reds", alpha=0.4)
    plt.title("Lipid Droplet Segmentation Overlay")
    plt.axis("off")
    plt.savefig(overlay_path, dpi=300, bbox_inches="tight")
    plt.close()

    return df_name

In [6]:
seg_droplets("./", "1-797.2nm-250mW-zoom6.0.tiff")

Total regions detected: 598
Regions after filtering: 59


'./roi1_lipid_props.csv'

### Overlay PRM result onto original image

In [20]:
from pathlib import Path
import numpy as np, cv2, tifffile as tiff

def make_overlays_and_ratios(directory, bg_id="10023", thr=230, alpha=0.5):
    d = Path(directory); out = d / "merged"; out.mkdir(exist_ok=True)
    keys = ["cholesterol_ester","cholesterol","dha_omega","omega","cardiolipin","pc","pe","tag"]
    colors = {"cardiolipin":(0,0,255),"cholesterol":(0,255,255),"cholesterol_ester":(0,165,255),
              "dha_omega":(255,255,0),"omega":(255,0,0),"pc":(255,0,255),"pe":(0,255,0),"tag":(128,0,128)}
    imgs = {k: [] for k in keys}
    bg8 = None

    # Load all .tif/.tiff; background as 16-bit -> scale to 8-bit
    for p in d.iterdir():
        if not p.is_file() or p.suffix.lower() not in {".tif",".tiff"}: continue
        name = p.name.lower()
        if bg_id in name:
            bg16 = cv2.imread(str(p), cv2.IMREAD_UNCHANGED)         # keep 16-bit
            if bg16.ndim == 3: bg16 = cv2.cvtColor(bg16, cv2.COLOR_BGR2GRAY)
            bg8 = (bg16.astype(np.float32) * (255.0/4095.0)).clip(0,255).astype(np.uint8)
            continue
        arr = cv2.imread(str(p), cv2.IMREAD_GRAYSCALE)
        if arr is None: continue
        for kk in keys:
            if kk in name:
                imgs[kk].append(arr.astype(np.float32))
                break

    if bg8 is None: print("[ERROR] Background not found."); return
    H, W = bg8.shape
    base = cv2.cvtColor(bg8, cv2.COLOR_GRAY2BGR).astype(np.float32)

    def overlay(mask, color_bgr):
        out = base.copy(); c = np.array(color_bgr, np.float32)
        out[mask] = (1 - alpha) * out[mask] + alpha * c
        return out.clip(0,255).astype(np.uint8)

    def save_ratio(num_key, den_key, fname):
        num_list = [x for x in imgs[num_key] if x.shape==(H,W)]
        den_list = [x for x in imgs[den_key] if x.shape==(H,W)]
        if not num_list or not den_list: return
        r = np.mean(num_list,0) / (np.mean(den_list,0) + 1e-6)
        r[r > 1.5] = 0.0
        tiff.imwrite(str(out / fname), r.astype(np.float32))

    # Overlays
    for k, arrs in imgs.items():
        if not arrs: continue
        mask = np.zeros((H, W), dtype=bool)
        for a in arrs:
            if a.shape==(H,W): mask |= (a > thr)
        cv2.imwrite(str(out / f"{k.capitalize()}_overlay.png"), overlay(mask, colors[k]))

    # Ratios
    save_ratio("cholesterol_ester","cholesterol","ester_ratio.tiff")
    save_ratio("dha_omega","omega","dha_ratio.tiff")

In [29]:
res = make_overlays_and_ratios("../PRM_SRS_out_viz/day35/cl/", thr=230, alpha=0.8)